# 导师制名企实训班商业智能方向 004期 Lesson 13

### Thinking 1: 线性规划与混合整数规划的区别是什么？

线性规划问题的目标函数及约束条件均为线性函数，对决策变量没有特殊要求，通常可用数学方法求解。混合整数规划要求部分决策变量为整数，所以通常使用启发式、遗传算法等搜索方法求解。

### Thinking 2: 针对VRP问题，常见的约束条件都有哪些？

* 车辆具有可携带的最大重量或数量
* 司机需要在指定时间窗口内访问某位置
* 点的访问顺序
* 多个车辆所运货物量是否均匀
* 车辆行驶最大距离等

### Action 1: Santa的接待安排  
圣诞节前100天，Santa开放了workshop，欢迎以家庭单位的参观，如何更合理的安排这些家庭参观？  
每个家庭有10个选择choice0-9，数字代表了距离圣诞节的天数，比如 1代表12月24日，每个家庭必须并且只安排一次参观  
家庭数量 5000，即family_id 为[0, 4999]，每天访问的人数需要在125-300人  
为了更合理的计算Santa的安排能力，我们使用preference cost和accounting penalty两个指标  
1）preference cost，代表Santa的个性化安排能力  
2）accounting penalty，代表Santa安排的财务成本  
每天接待的人员数N(d)如果大于125，就会拥挤，产生过多的清洁成本  
最终的 Score = preference cost + accounting penalty  
最终提交每个家庭的安排 submission.csv  


In [1]:
import numpy as np
import pandas as pd

### Step 1 ： 数据加载

In [2]:
# 数据加载
data = pd.read_csv('data/family_data.csv', index_col='family_id')
data

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


### Step 2 ：数据预处理
1. 计算Perference Cost矩阵 pcost_mat
2. 计算Accounting Cost矩阵 acost_mat
3. 计算每个家庭的人数 FAMILY_SIZE
4. 每个家庭的倾向选择（choice_） DESIRED


In [3]:
# n代表家庭成员个数，如果满足第choice需求，需要的penalty
def get_penalty(n, choice):
    if choice == 0:
        penalty = 0
    if choice == 1:
        penalty = 50
    if choice == 2:
        penalty = 50 + 9 * n
    if choice == 3:
        penalty = 100 + 9 * n
    if choice == 4:
        penalty = 200 + 9 * n
    if choice == 5:
        penalty = 200 + 18 * n
    if choice == 6:
        penalty = 300 + 18 * n
    if choice == 7:
        penalty = 300 + 36 * n
    if choice == 8:
        penalty = 400 + 36 * n
    if choice == 9:
        penalty = 500 + (36+199) * n
    if choice > 9 :
        penalty = 500 + (36+398) * n
    return penalty

In [4]:
# 安排的天数
N_DAYS = 100
# 家庭ID的个数
N_FAMILY = 5000
# 最小的承载量
MIN_OCCUPANCY = 125
# 最大的承载量
MAX_OCCUPANCY = 300

In [5]:
# 计算pcost_mat, 每个家庭，在什么时候（day 0-99）访问时的penalty
# 大小 5000 * 100的矩阵
pcost_mat = np.full(shape=(N_FAMILY,100), fill_value=999999)

for f in range(N_FAMILY):
    # 家庭成员数
    f_num = data.loc[f, 'n_people']
    # 对于第f个家庭，初始化pcost = other choice下的penalty
    pcost_mat[f,:] = get_penalty(f_num, 10)
    # 计算choice 0-9的 penalty 
    for choice in range(10):
        temp = data.loc[f][choice]
        penalty = get_penalty(f_num, choice)
        pcost_mat[f,temp-1] = penalty
pcost_mat

array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  616, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [6]:
pcost_mat.shape

(5000, 100)

In [7]:
# 计算accounting penalty矩阵，前一天的参观人数，当天的参观人数
acost_mat = np.zeros(shape=(MAX_OCCUPANCY+1, MAX_OCCUPANCY+1), dtype=np.float64)
# 当天安排的人数
for i in range(acost_mat.shape[0]):
    # 前一天安排的人数
    for j in range(acost_mat.shape[1]):
        diff=abs(i-j)
        acost_mat[i,j] = max(0, (i-125)/400*i**(0.5+diff/50.0))
acost_mat

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        7.46610759e+00, 8.36716954e+00, 9.37697794e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        8.43020770e+00, 7.52185316e+00, 8.43020770e+00],
       [5.52415954e+15, 4.92860244e+15, 4.39725208e+15, ...,
        9.51970597e+00, 8.49339085e+00, 7.57772228e+00]])

In [8]:
FAMILY_SIZE = data['n_people'].values
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4])

In [9]:
# DESIRED 代表每个家庭CHOICE-1
DESIRED = data.values[:,:-1]-1
DESIRED

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]])

### Step3，使用LP和MIP求解 规划方案
1. 先使用LP 对绝大部分家庭进行规划
2. 再使用MIP 对剩余家庭进行规划
3. 汇总两边的结果 => 最终规划方案


In [10]:
from ortools.linear_solver import pywraplp
# 线性规划,只以preference_cost为目标
def solveLP():
    # 线性规划 优化器
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    # family_id 在第j天是否参观
    x={}
    # 每一天有哪些家庭
    candidates = [[] for x in range(N_DAYS)]
    # family_id
    for i in range(N_FAMILY):
        # family_id 的choice
        for j in DESIRED[i,:]:
            # 在第j天，有第i个family参观
            candidates[j].append(i)
            # 定义决策变量 x[i, j] i代表family_id, j代表第j天参观
            x[i,j] = solver.BoolVar('x[%i,%i]' %(i,j))
    # 每台参观的人数 j代表100天
    daily_occupancy = [solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]]) \
                        for j in range(N_DAYS)]
    # 每个家庭，在10个choice中出现的情况
    family_presence = [solver.Sum(x[i,j] for j in DESIRED[i,:]) \
                for i in range(N_FAMILY)]
    # 定义目标函数 preference cost部分
    preference_cost = solver.Sum([pcost_mat[i,j] *x[i,j] for i in range(N_FAMILY) \
                                  for j in DESIRED[i,:]])
    # 满足preference_cost 最小
    solver.Minimize(preference_cost)
    # 人为增加的约束条件 Constraints
    for j in range(N_DAYS-1):
        # 当前人数不超过前一天人数+25
        solver.Add(daily_occupancy[j] - daily_occupancy[j+1]<=25)
        solver.Add(daily_occupancy[j+1] - daily_occupancy[j]<=25)
    # 每个家庭都在10个choice中出现1次
    for i in range(N_FAMILY):
        solver.Add(family_presence[i]==1)
    # 每天访问人数约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j]<=MAX_OCCUPANCY)
        
    result = solver.Solve()
    # 得到参观日安排
    temp = [(i,j,x[i,j].solution_value()) for i in range(N_FAMILY) \
        for j in DESIRED[i,:] if x[i,j].solution_value()>0]
    df = pd.DataFrame(temp, columns=['family_id','day','result'])
    return df

In [11]:
%%time
result = solveLP()
result

CPU times: user 10.7 s, sys: 27.9 ms, total: 10.8 s
Wall time: 10.7 s


,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5075,4995,15,1.0
5076,4996,87,1.0
5077,4997,31,1.0
5078,4998,91,1.0


In [12]:
result.value_counts()
# 设置阈值
THRS = 0.999
assigned_df = result[result.result > THRS]
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5075,4995,15,1.0
5076,4996,87,1.0
5077,4997,31,1.0
5078,4998,91,1.0


In [13]:
# 没有安排的，不为0和1
unassigned_df = result[(result.result<THRS) & (result.result>1-THRS)]
unassigned_df

,family_id,day,result
60,59,38,0.25
61,59,14,0.75
242,240,32,0.75
243,240,56,0.25
265,262,31,0.50
...,...,...,...
4990,4912,8,0.40
4992,4914,38,0.60
4993,4914,43,0.40
5040,4961,53,0.75


In [14]:
# 计算每天访问的人数(根据assigned_df)
assigned_df['family_size'] = FAMILY_SIZE[assigned_df.family_id]
assigned_df

/home/wll/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5075,4995,15,1.0,4
5076,4996,87,1.0,2
5077,4997,31,1.0,6
5078,4998,91,1.0,5


In [15]:
# 按照day进行聚合
occupancy = assigned_df.groupby('day').family_size.sum().values
occupancy

array([285, 271, 294, 293, 263, 242, 223, 247, 273, 297, 288, 292, 275,
       250, 245, 272, 292, 292, 271, 248, 223, 244, 264, 291, 292, 296,
       273, 249, 234, 251, 278, 283, 252, 235, 205, 184, 202, 233, 253,
       226, 210, 183, 204, 229, 247, 281, 256, 223, 204, 198, 222, 248,
       256, 223, 208, 185, 173, 196, 219, 198, 174, 141, 124, 121, 149,
       170, 178, 160, 136, 123, 125, 135, 158, 185, 158, 134, 121, 125,
       142, 167, 186, 167, 138, 120, 128, 155, 174, 203, 177, 152, 130,
       122, 130, 158, 183, 158, 128, 125, 122, 124])

In [16]:
min_occupancy = np.array([max(0, MIN_OCCUPANCY - x) for x in occupancy])
min_occupancy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 1])

In [17]:
max_occupancy = np.array([MAX_OCCUPANCY - x for x in occupancy])
max_occupancy

array([ 15,  29,   6,   7,  37,  58,  77,  53,  27,   3,  12,   8,  25,
        50,  55,  28,   8,   8,  29,  52,  77,  56,  36,   9,   8,   4,
        27,  51,  66,  49,  22,  17,  48,  65,  95, 116,  98,  67,  47,
        74,  90, 117,  96,  71,  53,  19,  44,  77,  96, 102,  78,  52,
        44,  77,  92, 115, 127, 104,  81, 102, 126, 159, 176, 179, 151,
       130, 122, 140, 164, 177, 175, 165, 142, 115, 142, 166, 179, 175,
       158, 133, 114, 133, 162, 180, 172, 145, 126,  97, 123, 148, 170,
       178, 170, 142, 117, 142, 172, 175, 178, 176])

In [18]:
# 使用整数规划进行求解
def solveIP(families, min_occupancy, max_occupancy):
    # 创建求解器
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    # 需要安排的家庭
    n_families = len(families)
    # family_id 在第j天是否参观
    x={}
    # 每一天有哪些家庭
    candidates = [[] for x in range(N_DAYS)]
    # family_id
    for i in families:
        # family_id 的choice
        for j in DESIRED[i,:]:
            # 在第j天，有第i个family参观
            candidates[j].append(i)
            # 定义决策变量 x[i, j] i代表family_id, j代表第j天参观
            x[i,j] = solver.BoolVar('x[%i,%i]' %(i,j))
    # 每台参观的人数 j代表100天
    daily_occupancy = [solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]]) \
                        for j in range(N_DAYS)]
    # 每个家庭，在10个choice中出现的情况
    family_presence = [solver.Sum(x[i,j] for j in DESIRED[i,:]) \
                for i in families]
    # 定义目标函数 preference cost部分
    preference_cost = solver.Sum([pcost_mat[i,j] *x[i,j] for i in families \
                                  for j in DESIRED[i,:]])
    # 满足preference_cost 最小
    solver.Minimize(preference_cost)
    # 人为增加的约束条件 Constraints
    for j in range(N_DAYS-1):
        solver.Add(daily_occupancy[j] - daily_occupancy[j+1]<=25)
        solver.Add(daily_occupancy[j+1] - daily_occupancy[j]<=25)
    # 每个家庭都在10个choice中出现1次
    for i in range(n_families):
        solver.Add(family_presence[i]==1)
    # 每天访问人数约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=min_occupancy[j])
        solver.Add(daily_occupancy[j]<=max_occupancy[j])
        
    result = solver.Solve()
    # 得到参观日安排
    temp = [(i,j) for i in families \
        for j in DESIRED[i,:] if x[i,j].solution_value()>0]
    df = pd.DataFrame(temp, columns=['family_id','day'])
    return df

In [19]:
%%time
# 没有安排的family_id
unassigned = unassigned_df.family_id.unique()
result = solveIP(unassigned,min_occupancy, max_occupancy) 
result

CPU times: user 115 ms, sys: 4 ms, total: 119 ms
Wall time: 118 ms


,family_id,day
0,59,38
1,240,32
2,262,31
3,488,39
4,500,26
...,...,...
64,4850,4
65,4886,98
66,4912,17
67,4914,38


#### 合并结果

In [20]:
df = pd.concat((assigned_df[['family_id', 'day']],result)).sort_values('family_id')
df

,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5075,4995,15
5076,4996,87
5077,4997,31
5078,4998,91


### Step4, 结果评估
按照evaluation标准，计算  
Score = preference cost + accounting penalty

In [21]:
from numba import njit


# 根据安排情况，计算这个安排的preference cost
@njit(fastmath=True)
def pcost(prediction):
    daily_occupancy = np.zeros(N_DAYS+1, dtype=np.int64)
    penalty = 0
    for (i,p) in enumerate(prediction):
        # 计算家庭人数
        n = FAMILY_SIZE[i]
        # 第i个家庭，p天访问时的cost
        penalty += pcost_mat[i,p]
        # 计算当天的人数
        daily_occupancy[p] += n
    return penalty, daily_occupancy


# 根据安排情况，计算安排的accounting cost
@njit(fastmath=True)
def acost(daily_occupancy):
    accounting_cost = 0
    num_out_of_range = 0
    daily_occupancy[-1] = daily_occupancy[-2]
    for day in range(N_DAYS):
        # 前一天
        n_p1 = daily_occupancy[day+1]
        # 当天
        n = daily_occupancy[day]
        # 如果超过了承载范围，则设置 out_of_range
        num_out_of_range += (n>MAX_OCCUPANCY) or (n<MIN_OCCUPANCY)
        # 计算accouting cost
        accounting_cost += acost_mat[n,n_p1]
    return accounting_cost, num_out_of_range

# 根据安排prediction
@njit(fastmath=True)
def cost_function(prediction):
    # 基于prediction，计算preference cost和accouting cost
    # 统计perference cost和每天承载数量
    penalty, daily_occupancy = pcost(prediction)
    # 根据每天承载数量，计算accouting cost
    accounting_cost, num_out_of_range = acost(daily_occupancy)
    final_score = penalty+accounting_cost+num_out_of_range *99999999
    return final_score

In [22]:
prediction = df.day.values
cost_function(prediction)

77480.2439961357

In [23]:
def save_result(pred, filename):
    result = pd.DataFrame(range(N_FAMILY), columns=['family_id'])
    result['assigned_day'] = pred + 1
    result.to_csv(filename, index=False)
    print(filename+' saved')
    return result

In [24]:
result = save_result(prediction, 'submission1.csv')

submission1.csv saved


### Step5，方案优化
通过更换family_id的选择，查找更好的score  
每次更换后，都对方案进行评估，选择更小的score方案

In [25]:
def find_better(pred):
    fobs = np.argsort(FAMILY_SIZE)
    score = cost_function(pred)
    # 打擂法
    original_score = np.inf
    # 如果找不到更新， 则退出
    while score<original_score:
        original_score = score
        for family_id in fobs:
            for pick in range(10):
                # 得到family_id在choice pick的日期 day
                day = DESIRED[family_id, pick]
                # 改family的原有日期oldvalue
                oldvalue = pred[family_id]
                # 将原有oldvalue替换为现在的choice pick
                pred[family_id] = day
                # 重新计算分数
                new_score = cost_function(pred)
                # 如果比原来分数小，更新choice成功
                if new_score<score:
                    score = new_score
                else:
                    # 设置为原来的oldvalue
                    pred[family_id] = oldvalue
        print(score, end='\r')
    print(score)


In [26]:
%%time
new = prediction.copy()
find_better(new)

-9.991515420421351e+246
CPU times: user 1.6 s, sys: 7.98 ms, total: 1.61 s
Wall time: 1.6 s


In [27]:
result = save_result(new, 'submission2.csv')

submission2.csv saved


### Action 2: 多辆车的路径规划 VRP：
条件：经过中国33个城市，一共4辆车，每辆车最大行驶10000公里  
目标：使得每辆车的行驶里程数更接近  
需要注意：  
1）在VRP问题中，路径上给点赋的index和点实际的index不一样，需要使用IndexToNode方法进行转换才能得到实际的index  


In [28]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [29]:
df = pd.read_excel('data/cities.xlsx')
df.head()

,name,location
0,北京,"116.403039,39.914909"
1,天津,"117.216419,39.143831"
2,哈尔滨,"126.637622,45.766187"
3,长春,"125.330606,43.918866"
4,沈阳,"123.401674,41.800522"


In [30]:
# 设置数据，类型为字典格式
def create_data_model(num_vehicles=1):
    data = {}
    temp = pd.read_excel('data/distance.xlsx', index_col=0)
    # 城市之间的距离
    data['distance_matrix'] = temp.values / 1000
    # 车的数量
    data['num_vehicles'] = num_vehicles
    # 原点下标
    data['depot'] = 0
    return data

In [32]:
# 计算结果
def get_solution(manager, routing, solution, num_vehicles):
    # 记录每辆车的里程 
    distance_list = []
    # 记录每辆车的路径规划
    route_list = []
    # 遍历每辆车
    for vehicle_id in range(num_vehicles):
        # 初始化距离为0
        route_distance = 0
        # 针对第vehicle_id 车辆进行规划
        route = []
        # Start后面是vehicle_id
        # 从vehicle_id的起始节点出发
        index = routing.Start(vehicle_id)
        # 判断这辆车的路径是否结束
        while not routing.IsEnd(index):
            # 使用IndexToNode将manager中的index装换为distance_matrix中的index
            index_show = manager.IndexToNode(index)
            # 添加到route
            route.append(index_show)
            previous_index = index
            # 走到下一个节点
            index = solution.Value(routing.NextVar(index))
            # 针对vehicle=0，统计从previous_index到index节点的距离
            route_distance += routing.GetArcCostForVehicle(
                previous_index, 
                index, 
                0)
        route_list.append(route)
        distance_list.append(route_distance)
    return route_list, distance_list

In [33]:
# 添加距离约束
def add_distance_dimension(routing, transit_callback_index):
    dimension_name = 'Distance'
    routing.AddDimension(transit_callback_index,
                         0,  # no slack
                         10000,  # 车辆最大行驶距离
                         True,  # start cumul to zero
                         dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    # 尽量减少车辆之间的最大距离, spancost的系数为100
    distance_dimension.SetGlobalSpanCostCoefficient(100)

In [34]:
# 求解
def work(num_vehicles):
    # Step1，初始化数据
    data = create_data_model(num_vehicles)
    # Step2, 创建路线管理，tsp_size（城市数量）, num_vehicles（车的数量）, depot（原点）
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # Step3, 创建 Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    # Step4, 计算两点之间的距离
    def distance_callback(from_index, to_index):
        # 将index转换为distance_matrix中的节点index
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    # 注册距离函数
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # 添加距离维度
    add_distance_dimension(routing, transit_callback_index)
    # Step5, 设置参数策略 Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    # 定义最优化问题的目标
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # Step6，求解路径规划
    solution = routing.SolveWithParameters(search_parameters)
    # Step7，输出结果
    return get_solution(manager, routing, solution, data['num_vehicles'])

In [35]:
%%time
route_list, distance_list = work(num_vehicles=4)

CPU times: user 467 ms, sys: 20 µs, total: 467 ms
Wall time: 466 ms


In [36]:
route_list

[[0, 7, 18, 19, 17, 22, 6],
 [0, 21],
 [0, 5, 20, 23, 24, 26, 27, 25, 12, 32, 11, 31, 30, 13, 16],
 [0, 14, 28, 29, 10, 9, 8, 15, 1, 4, 2, 3]]

In [37]:
distance_list

[6341, 7096, 6749, 6845]